In [1]:
from fn import op, _
from itertools import repeat
from functools import reduce
# from objproxies import LazyProxy
import json
from copy import deepcopy, copy
from pipetools import pipe
from functools import partial

In [2]:
def last_index(l):
    return len(l)-1
def retrieve_state(l, offset):
    return l[last_index(l) + offset + 1]
def bound_norm_random(low, high):
    res = np.random.normal((high+low)/2,(high-low)/6)
    if (res<low or res>high):
        res = bound_norm_random(low, high)
    return res

In [3]:
# def pipeline(*steps):
#     return reduce(lambda x, y: y(x), list(steps))
# def compose(*funcs):
#     return lambda x: reduce(lambda f, g: g(f), list(funcs), x)

In [4]:
# ToDo:
# Handle case where Mechanisms have no input. Perhaps the sentinel value of 0
# Mock Simulation SimCADUI Lucid chart 

In [5]:
# UI Behaviors per Mechanism
def b1m1(step, sL, s):
    return s['s1']
def b2m1(step, sL, s):
    return s['s1'] * s['s2']

def b1m2(step, sL, s):
    return s['s1']
def b2m2(step, sL, s):
    return s['s1'] / s['s2']

def b1m3(step, sL, s):
    return s['s1']
def b2m3(step, sL, s):
    ps = retrieve_state(sL, -3)
    return ps['s2']

# UI States per Mechanism
def s1m1(step, sL, s, _input):
    s['s1'] = s['s1']**2 + _input
def s2m1(step, sL, s, _input):
    s['s2'] = s['s2'] + 1 + _input


def s3m1(step, sL, s, _input):
    s['s3'] = s['s3']

def s1m2(step, sL, s, _input):
    s['s1'] = s['s1'] + _input
def s2m2(step, sL, s, _input):
    s['s2'] = s['s2']
    
def s3m2(step, sL, s, _input):
    s['s3'] = s['s3'] + 1
    
def s1m3(step, sL, s, _input):
    s['s1'] = s['s1']
def s2m3(step, sL, s, _input):
    s['s2'] = s['s2'] + _input
    
state_dict = {
    's1': 2,
    's2': 4,
    's3': 300
}
    
j = {
    "m1": {
        "behaviors": {
            "b1": b1m1,
            "b2": b2m1
        },
        "states": {
            "s1": s1m1,
            "s2": s2m1,
            "s3": s3m1
        }
    },
    "m2": {
        "behaviors": {
            "b1": b1m2,
            "b2": b2m2
        },
        "states": {
            "s1": s1m2,
            "s2": s2m2,
            "s3": s3m2
        }
    },
    "m3": {
        "behaviors": {
            "b1": b1m3,
            "b2": b2m3
        },
        "states": {
            "s1": s1m3,
            "s2": s2m3
        }
    }
}

In [6]:
def generate_configs(j):
    return list(
        map(
            lambda x: (
                list(j[x]["states"].values()),
                list(j[x]["behaviors"].values())
            ), 
            j.keys()
        )
    )

# partials = list(map(lambda f: partial(f, step, sL), funcs))
def getColResults(step, sL, s, funcs):
    return list(map(lambda f: f(step, sL, s), funcs))

def getBehaviorInput(step, sL, s, funcs): 
    return op.foldr(_ + _)(getColResults(step, sL, s, funcs))

def mech_step(step, sL, state_funcs, behavior_funcs):
    # Purge Memory accociated with certain objs
    # Truncate reffs
    in_copy, mutatable_copy, out_copy = deepcopy(sL), deepcopy(sL), deepcopy(sL)
    last_in_obj, last_mut_obj = in_copy[-1], mutatable_copy[-1]
    
    _input = getBehaviorInput(step, sL, last_in_obj, behavior_funcs)

    for f in state_funcs:
        f(step, sL, last_mut_obj, _input)
        
    last_mut_obj["step"] = step
    out_copy.append(last_mut_obj) 
    return out_copy

# isolate ALL step logic within 'step_func'
def pipeline(states_list, configs, step_func):
    step = 0
    genesis_states = states_list[-1]
    genesis_states['step'] = step
    states_list = [genesis_states]
    
    step = step_func(step, 1)
    for config in configs:
        s_conf, b_conf = config[0], config[1]
        states_list = mech_step(step, states_list, s_conf, b_conf)
        
        step = step_func(step, 1)
    return states_list

In [7]:
states_list = [state_dict]
states_list

[{'s1': 2, 's2': 4, 's3': 300}]

In [8]:
configs = generate_configs(j)
def step_func(x, y):
    return x + y
pipeline(states_list, configs, step_func)
# categorize by simulation

[{'s1': 2, 's2': 4, 's3': 300, 'step': 0},
 {'s1': 14, 's2': 15, 's3': 300, 'step': 1},
 {'s1': 28.933333333333334, 's2': 15, 's3': 301, 'step': 2},
 {'s1': 28.933333333333334, 's2': 47.93333333333334, 's3': 301, 'step': 3}]

In [9]:
15 + 28.933333333333334 + 4

47.93333333333334

In [10]:
# def multiply(x,y,z):
#         return x * y * z

# # create a new function that multiplies by 2
# dbl = partial(partial(multiply,2), multiply, 2)
# print(dbl(2))